## Imports

In [ ]:
%%writefile requirements.txt

lightgbm==4.5.0
xgboost==2.1.1
scikit-learn=1.4.2


In [3]:
%%capture

!pip install -r requirements.txt

In [1]:
%%writefile -a myimports.py

print(f"\n---> Commencing Imports")

from gc import collect
from warnings import filterwarnings
filterwarnings('ignore')
from IPython.display import display_html, clear_output
clear_output()
import os, sys, logging, re, joblib, ctypes, shutil
from copy import deepcopy

# General library imports
from os import path, walk, getpid
from psutil import Process
from collections import Counter
from itertools import product
import ctypes
libc=ctypes.CDLL("libc.so.6")

from IPython.display import display_html, clear_output
from pprint import pprint
from functools import partial
from copy import deepcopy
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from numpy.typing import ArrayLike, NDArray
import polars as pl
import polars.selectors as cs
from polars.testing import assert_frame_equal

import matplotlib.pyplot as plt
import seaborn as sns
from colorama import Fore, Style, init
from tqdm.notebook import tqdm


Appending to myimports.py


In [2]:
%%writefile -a myimports.py

# Importing model and pipeline specifics
from category_encoders import OrdinalEncoder, OneHotEncoder

# Pipeline specifics
from sklearn.preprocessing import (RobustScaler,
MinMaxScaler,
StandardScaler,
FunctionTransformer as FT,
PowerTransformer,
)
from sklearn.impute import SimpleImputer as SI
from sklearn.model_selection import (
    RepeatedStratifiedKFold as RSKF,
    StratifiedKFold as SKF,
    KFold,
    GroupKFold as GKF,
    RepeatedKFold as RKF,
    PredefinedSplit as PDS,
    cross_val_score,
    cross_val_predict
)
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import VarianceThreshold as VT
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import (BaseEstimator,TransformerMixin, RegressorMixin, clone)
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.linear_model import Ridge
from sklearn.metrics import ( 
    root_mean_squared_error as rmse,
    mean_absolute_error as mae
    )
from sklearn.ensemble import (
    RandomForestRegressor as RFR,
    ExtraTreesRegressor as ETR,
    HistGradientBoostingRegressor as HGBR)

# Importing model packages
import xgboost as xgb, lightgbm as lgb
from xgboost import XGBRegressor as XGBR, QuantileDMatrix
from lightgbm import log_evaluation, early_stopping, LGBMRegressor as LGBMR
from catboost import CatBoostRegressor as CBR, Pool

# Importing ensemble and tuning packages
import optuna
from optuna import Trial, trial, create_study
from optuna.pruners import HyperbandPruner
from optuna.samplers import TPESampler, CmaEsSampler
optuna.logging.disable_default_handler()


Appending to myimports.py


In [3]:
%%writefile -a myimports.py

# setting rc parameters in seaborn for plots and graphs
sns.set_theme(
    {
        "axes.facecolor":"#ffffff",
        "figure.facecolor":"#ffffff",
        "axes.edgecolor": "#000000",
        "grid.color":"#ffffff",
        "font.family":["Cambria"],
        "axes.labelcolor":"#000000",
        "xtick.color":"#000000",
        "ytick.color":"#000000",
        "grid.linewidth":0.75,
        "grid.linestyle":"--",
        "axes.titlecolor":"#0099e6",
        "axes.titlesize":8.5,
        "axes.labelweight":"bold",
        "legend.fontsize":7.0,
        "legend.title_fontsize":7.0,
        "font.size":7.5,
        "xtick.labelsize":7.5,
        "ytick.labelsize":7.5
    }
        )

# color printing
def PrintColor(text:str,color=Fore.BLUE,style=Style.BRIGHT):
    print(style+color+text+Style.RESET_ALL)

# Checking package versions
import xgboost as xgb, lightgbm as lgb, catboost as cb, sklearn as sk, pandas as pd, polars as pl
print(f"---> XGBoost = {xgb.__version__} | LightGBM = {lgb.__version__} | CatBoost = {cb.__version__}")
print(f"---> Sklearn = {sk.__version__} | Pandas = {pd.__version__} | Polars = {pl.__version__}")

Appending to myimports.py


In [4]:
%%writefile -a myimports.py
 
class MyLogger:
    """
    This class helps to supress logs in lightgbm and optuna
    """
    def init(self,logging_lbl:str):
        self.logger=logging.getLogger(logging_lbl)
        self.logger.setLevel(logging.ERROR)

    def info(self,message):
        pass

    def warning(self,message):
        pass

    def error(self,message):
        self.logger.error(message)

# Customizing logging for xgboost

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logger=logging.getLogger(__name__)
logger.setLevel(logging.ERROR)
formatter=logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')

stdout_handler=logging.StreamHandler(sys.stdout)
stdout_handler.setLevel(logging.INFO)
stdout_handler.setFormatter(formatter)

file_handler=logging.FileHandler(f"xgb_optimize.log")
file_handler.setLevel(logging.ERROR)
file_handler.setFormatter(formatter)

logger.addHandler(file_handler)
logger.addHandler(stdout_handler)

class XGBLogging(xgb.callback.TrainingCallback):
    """
    This class is design for custom logging in xgboost
    This is to be used inside XGBoost callback
    """
    def __init__(self,epoch_log_interval=100):
        self.epoch_log_interval=epoch_log_interval

    def after_interaction(
            self,model,epoch:int,evals_log:xgb.callback.TrainingCallback.EvalsLog
    ):
        if self.epoch_log_interval<=0:
            pass

        elif (epoch%self.epoch_log_interval==0):
            for data,metric in evals_log.items():
                for metric_name, log in metric.items():
                    score=log[-1][0] if isinstance(log[-1], tuple) else log[-1]
                    logger.info(f"XGBLogging epoch {epoch} dataset {data} {metric_name} {score}")

        return False







Appending to myimports.py


In [5]:
%%time
exec(open('myimports.py','r').read())
print()


---> XGBoost = 2.1.1 | LightGBM = 4.5.0 | CatBoost = 1.2.7
---> Sklearn = 1.5.2 | Pandas = 2.2.3 | Polars = 1.8.2

CPU times: user 3.01 s, sys: 1.16 s, total: 4.17 s
Wall time: 3.9 s


In [8]:
%%writefile -a training.py

# Configuration class
class CFG:
    """
    Configuration class for parameters and CV strategy for tuning and training
    """

    # Data preparation
    version_nb         =1
    model_id           ="V1_2"
    model_label        ="ML"

    test_req           =False
    test_sample_frac   =1000

    gpu_switch         ="OFF"
    state              =42
    target             =f"bg+1:00"
    grouper            =f"p_num"

    ip_path            =f'/mnt/c/Users/anush/Documents/Competitions/BrisT1D Blood Glucose Prediction/inputs'
    op_path            =f'/mnt/c/Users/anush/Documents/Competitions/BrisT1D Blood Glucose Prediction/outputs'

    # Model Training
    pstprcs_oof        =False
    pstprcs_train      =False
    pstprcs_test       =False
    ML                 =True
    test_preds_req     =True

    pseudo_lbl_req      =False
    pseudolbl_up        =0.975
    pseudolbl_low       =0.0

    n_splits        =3 if test_req else 5
    n_repeats       =1
    nbrnd_erly_stp  =100
    mdlcv_mthd      ='GKF'

    #Ensemble
    ensemble_req=True
    metric_obj='minimize'

    # Global variables for plotting
    grid_specs={
        'visible':True,
        'which':"both",
        "linestyle":"--",
        "color":"lightgrey",
        "linewidth":0.75
        }
    
    title_specs={
        'fontsize':9,
        'fontweight':"bold",
        'color':"#992600"
                 }
cv_selector=\
{
    "RKF":RKF(n_splits=CFG.n_splits,n_repeats=CFG.n_repeats,random_state=CFG.state),
    "RSKF":RSKF(n_splits=CFG.n_splits,n_repeats=CFG.n_repeats,random_state=CFG.state),
    "SKF":SKF(n_splits=CFG.n_splits,shuffle=True,random_state=CFG.state),
    "KF":KFold(n_splits=CFG.n_splits,shuffle=True,random_state=CFG.state),
    "GKF":GKF(n_splits=CFG.n_splits)
}

PrintColor(f"\n---> Configuration done !\n")
collect()


Appending to training.py


## PreProcessing


In [9]:

%%writefile -a training.py

def XformCols(df:pd.DataFrame):
    """
    This function do the following.
    * Create time components on time column
    * Removes special characters in column labels

    """

    df["hour_nb"]   =pd.to_datetime(df['time']).dt.hour
    df["minute_nb"] =pd.to_datetime(df['time']).dt.minute
    df.columns      =df.columns.str.replace("\W",'',regex=True)

    return df.drop("time", axis=1,errors='ignore')



Appending to training.py


In [10]:
%%capture

exec(open('training.py','r').read())

train=pd.read_csv(os.path.join(CFG.ip_path,'train.csv'),index_col="id",low_memory=False)
test=pd.read_csv(os.path.join(CFG.ip_path,'test.csv'),index_col="id",low_memory=False)
sub_f1=pd.read_csv(os.path.join(CFG.ip_path,'sample_submission.csv'),index_col="id")

if CFG.test_req:
    train=train.groupby(CFG.grouper).sample(n=CFG.test_sample_frac)

train.index=range(len(train))

_=utils.CleanMemory()

NameError: name 'utils' is not defined

In [19]:
%%time

PrintColor(f"---> Shapes= {train.shape} {test.shape} {sub_f1.shape}")

Xtrain=train.drop([CFG.target,CFG.grouper],axis=1, errors="ignore")
ytrain=train[CFG.target]
ygrp=train[CFG.grouper]
Xtest=test.drop([CFG.target,CFG.grouper],axis=1,errors="ignore")

# Defining the data pipeline
xform=Pipeline(
    [
        ("XformCols",FT(XformCols)),
        ("Xform",ColumnTransformer([
            ("SI",SI(strategy="mean"),make_column_selector(dtype_include=np.number)),
            ("OE",make_pipeline(*[SI(strategy="constant",fill_value="missing"),OrdinalEncoder(return_df=False)]),make_column_selector(pattern="activity"))
                                    ],
                                    remainder="passthrough",
                                    verbose_feature_names_out=False,))
    ],verbose=False).set_output(transform="pandas")

Xtrain=xform.fit_transform(Xtrain,ytrain)
Xtest=xform.transform(Xtest)

PrintColor(f"---> Shapes= {Xtrain.shape} {ytrain.shape} {Xtest.shape} {sub_f1.shape}")

# Initializing CV scheme
cv=cv_selector[CFG.mdlcv_mthd]

# Initializing CV folds across training data
folds=np.zeros(len(Xtrain))
for fold_nb,(train_idx,dev_idx) in enumerate(cv.split(Xtrain,ytrain,ygrp)):
    folds[dev_idx]=fold_nb
Xtrain["fold_nb"]=folds
del folds

# Adding source column for training algorithm
Xtrain["Source"]="Competition"

PrintColor(f"\nData Pipeline Strcuture\n",color=Fore.RED)
display(xform)

PrintColor(f"---> Shapes={Xtrain.shape} {ytrain.shape} {Xtest.shape} {sub_f1.shape}")
# PrintColor(python_utils.CleanMemory())

---> Shapes= (177024, 507) (3644, 506) (3644, 1)
---> Shapes= (177024, 506) (177024,) (3644, 506) (3644, 1)

Data Pipeline Strcuture



Pipeline(steps=[('XformCols',
                 FunctionTransformer(func=<function XformCols at 0x7fe3443ac400>)),
                ('Xform',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('SI', SimpleImputer(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fe33f34e750>),
                                                 ('OE',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinalencoder',
                                                                   OrdinalEncoder(return_df=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fe3af7b92e0>)],
                                   verbose_feature_names_out=False))])

---> Shapes=(177024, 508) (177024,) (3644, 506) (3644, 1)
CPU times: user 8.3 s, sys: 1.52 s, total: 9.82 s
Wall time: 9.79 s


GroupKFold(n_splits=5)